In [19]:
import cirq
import numpy as np
from typing import List, Tuple

def bloch_state_to_unitary(theta: float, phi: float) -> np.ndarray:
    """
    Convert Bloch sphere angles to the unitary that rotates |0⟩ to |θ,φ⟩.
    
    The state |θ,φ⟩ = cos(θ/2)|0⟩ + e^(iφ)sin(θ/2)|1⟩
    """
    # This is simply Rz(phi) @ Ry(theta)
    return np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.exp(1j*phi)*np.sin(theta/2), np.exp(1j*phi)*np.cos(theta/2)]
    ])

def find_best_clifford_t_approximation(angle: float, max_depth: int = 10) -> List[cirq.Gate]:
    """
    Find the best Clifford+T approximation for Rz(angle).
    Uses the fact that T = Rz(π/4) and S = Rz(π/2).
    """
    # T gate: e^(iπ/8) Rz(π/4)
    # We want to find k such that k*π/4 ≈ angle (mod 2π)
    
    # Normalize angle to [0, 2π)
    angle = angle % (2 * np.pi)
    
    # Find best approximation using denominator 2^max_depth
    denominator = 2 ** max_depth
    
    # Find k such that k * 2π / denominator ≈ angle
    k = round(angle * denominator / (2 * np.pi))
    k = k % denominator
    
    # Now express k in base 2 to minimize gates
    # We build the rotation from T and S gates
    gates = []
    
    # Convert k to a sum of powers of 2
    # Each T gate contributes π/4, each S contributes π/2
    # We want k * (2π / 2^max_depth) = k * π / 2^(max_depth-1)
    
    # Simplify: express in terms of π/4 units
    # k * (2π / 2^max_depth) = (k / 2^(max_depth-3)) * π/4
    
    n_units = k * (2 ** (3 - max_depth))  # Number of π/4 rotations
    n_units = round(n_units * (2 ** (max_depth - 3)))
    
    # Optimize using S gates (S = T^2)
    # Count in units of π/4
    total_quarter_turns = k % (denominator)
    
    # Express in minimal form
    # π/2 = 2 * π/4, so use S when possible
    n_half_turns = (total_quarter_turns * 8 // denominator) % 4
    remaining = (total_quarter_turns * 8 % denominator)
    
    n_quarter_turns = (remaining * 2 // denominator) % 2
    
    # Add S gates (Rz(π/2))
    for _ in range(n_half_turns):
        gates.append(cirq.S)
    
    # Add T gates (Rz(π/4))
    for _ in range(n_quarter_turns):
        gates.append(cirq.T)
    
    return gates

def create_bloch_state_clifford_t_simple(theta: float, phi: float, max_depth: int = 10) -> cirq.Circuit:
    """
    Create a circuit that prepares |θ,φ⟩ = Rz(φ) Ry(θ) |0⟩
    using a straightforward decomposition.
    """
    qubit = cirq.LineQubit(0)
    circuit = cirq.Circuit()
    
    # Strategy: Use exact Ry(theta) and approximate Rz(phi)
    # OR: Approximate both rotations
    
    # For better results, let's use Cirq's optimizer with Clifford+T gateset
    # First create the ideal circuit
    circuit.append(cirq.ry(theta)(qubit))
    circuit.append(cirq.rz(phi)(qubit))
    
    return circuit

def create_bloch_state_with_decomposition(theta: float, phi: float, 
                                          t_count: int = 10) -> cirq.Circuit:
    """
    Better approach: Use Cirq's built-in gate compilation to Clifford+T.
    
    Args:
        theta: Polar angle
        phi: Azimuthal angle  
        t_count: Approximate number of T gates to use (controls precision)
    """
    qubit = cirq.LineQubit(0)
    
    # Create ideal circuit
    ideal_circuit = cirq.Circuit()
    ideal_circuit.append(cirq.ry(theta)(qubit))
    ideal_circuit.append(cirq.rz(phi)(qubit))
    
    # Use Cirq's compiler to convert to specific gateset
    # Define Clifford+T gateset
    gateset = cirq.Gateset(cirq.H, cirq.S, cirq.T, cirq.CNOT, cirq.X, cirq.Y, cirq.Z)
    
    # Compile the circuit - this uses Solovay-Kitaev internally
    try:
        compiled = cirq.optimize_for_target_gateset(
            ideal_circuit,
            gateset=gateset,
            max_num_passes=5
        )
        return compiled
    except:
        # If compilation fails, use manual decomposition
        return manual_clifford_t_decomposition(theta, phi, t_count)

def manual_clifford_t_decomposition(theta: float, phi: float, precision_bits: int = 8) -> cirq.Circuit:
    """
    Manual decomposition with better approximation.
    """
    qubit = cirq.LineQubit(0)
    circuit = cirq.Circuit()
    
    # Decompose Ry(theta) into Clifford+T
    # Use: Ry(θ) = Rz(-π/2) Rx(θ) Rz(π/2) and Rx(θ) = H Rz(θ) H
    
    # First, apply Ry(theta)
    # Ry can be expressed as: S† H S† Rz(θ) S H S
    # But simpler: just use H and Rz
    
    # Approximate Ry(theta):
    # Ry(θ) ≈ H Rz(θ) H (not exact, but close for small theta)
    # Better: Ry(θ) = exp(iπ/4 Y) exp(-iπ/4 X) Rz(θ) exp(iπ/4 X) exp(-iπ/4 Y)
    
    # Even better: use the circuit Rz(φ/2) Rx(θ) Rz(φ/2 + π/2)
    # Where Rx(θ) = H Rz(θ) H
    
    # Most straightforward for Bloch state:
    # |θ,φ⟩ requires: First Ry(θ), then Rz(φ)
    
    # Ry(θ) using Clifford+H+Rz:
    circuit.append(cirq.S(qubit) ** -1)  # Rz(-π/2)
    circuit.append(cirq.H(qubit))
    
    # Approximate Rz(θ) with T gates
    rz_gates = approximate_rz_with_precision(theta, precision_bits)
    for gate in rz_gates:
        circuit.append(gate(qubit))
    
    circuit.append(cirq.H(qubit))
    circuit.append(cirq.S(qubit))  # Rz(π/2)
    
    # Now apply Rz(φ)
    rz_phi_gates = approximate_rz_with_precision(phi, precision_bits)
    for gate in rz_phi_gates:
        circuit.append(gate(qubit))
    
    return circuit

def approximate_rz_with_precision(angle: float, precision_bits: int) -> List[cirq.Gate]:
    """
    Approximate Rz(angle) using T and S gates.
    precision_bits controls accuracy: higher = more accurate
    """
    gates = []
    
    # Normalize to [0, 2π)
    angle = angle % (2 * np.pi)
    
    # Discretize: we can make rotations of 2π/2^n where n is precision
    n_steps = 2 ** precision_bits
    step_size = 2 * np.pi / n_steps
    
    # Find closest multiple
    n = round(angle / step_size) % n_steps
    
    # Express n in terms of T (π/4) and S (π/2) gates
    # Each step is 2π/2^precision_bits = π/2^(precision_bits-1)
    
    # T gate = Rz(π/4) = Rz(2π/8)
    # S gate = Rz(π/2) = Rz(2π/4)
    
    # We want n * (2π / 2^precision_bits)
    # Express as multiples of π/4
    
    quarter_turns = (n * 8) // n_steps
    
    # Use S and T optimally
    half_turns = quarter_turns // 2
    remaining_quarter = quarter_turns % 2
    
    # Optimize S gates (mod 4 since 4*S = I up to global phase)
    half_turns = half_turns % 4
    
    for _ in range(half_turns):
        gates.append(cirq.S)
    for _ in range(remaining_quarter):
        gates.append(cirq.T)
    
    return gates

# Example usage
if __name__ == "__main__":
    for theta in np.arange(0,np.pi,0.1):
        for phi in np.arange(0,np.pi,0.1):
            
            print(f"Creating Bloch sphere state |θ={theta:.4f}, φ={phi:.4f}⟩")
            #print(f"State: cos({theta/2:.4f})|0⟩ + e^(i*{phi:.4f})sin({theta/2:.4f})|1⟩\n")
            
            # Target state
            target_state = np.array([
                np.cos(theta/2),
                np.exp(1j * phi) * np.sin(theta/2)
            ])
            
            #print("Target state:", target_state)
            #print()
            
            # Try different precision levels
            for precision in [25]:
                #print(f"\n{'='*60}")
                #print(f"Precision bits = {precision}")
                #print(f"{'='*60}")
                
                circuit = manual_clifford_t_decomposition(theta, phi, precision_bits=precision)
                
                #print(f"Total gates: {len(circuit)}")
                
                # Count T gates
                t_count = sum(1 for moment in circuit for op in moment 
                            if op.gate == cirq.T or (isinstance(op.gate, cirq.ZPowGate) and abs(op.gate.exponent - 0.25) < 1e-10))
                #print(f"T-count: {t_count}")
                
                # Simulate
                simulator = cirq.Simulator()
                result = simulator.simulate(circuit)
                
                fidelity = abs(np.dot(np.conj(target_state), result.final_state_vector))**2
                print(f"Fidelity: {fidelity:.10f}")
                
                #print("\nCircuit:")
                #print(circuit)
                #print("\nFinal state:", result.final_state_vector)

Creating Bloch sphere state |θ=0.0000, φ=0.0000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=0.1000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=0.2000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=0.3000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=0.4000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=0.5000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=0.6000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=0.7000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=0.8000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=0.9000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=1.0000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=1.1000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=1.2000⟩
Fidelity: 1.0000000000
Creating Bloch sphere state |θ=0.0000, φ=1.3000⟩
Fidelity: 1.000

In [23]:
import numpy as np
from typing import Dict, Tuple

def create_bloch_state(theta: float, phi: float) -> np.ndarray:
    """
    Create the Bloch sphere state |θ,φ⟩.
    
    |θ,φ⟩ = cos(θ/2)|0⟩ + e^(iφ)sin(θ/2)|1⟩
    
    Args:
        theta: Polar angle (0 to π)
        phi: Azimuthal angle (0 to 2π)
    
    Returns:
        State vector as numpy array
    """
    state = np.array([
        np.cos(theta/2),
        np.exp(1j * phi) * np.sin(theta/2)
    ], dtype=complex)
    
    return state

def get_clifford_basis_states() -> Dict[str, np.ndarray]:
    """
    Return the 6 Clifford basis states:
    |0⟩, |1⟩, |+⟩, |-⟩, |i⟩, |-i⟩
    """
    states = {
        '|0⟩': np.array([1, 0], dtype=complex),
        '|1⟩': np.array([0, 1], dtype=complex),
        '|+⟩': np.array([1, 1], dtype=complex) / np.sqrt(2),
        '|-⟩': np.array([1, -1], dtype=complex) / np.sqrt(2),
        '|i⟩': np.array([1, 1j], dtype=complex) / np.sqrt(2),
        '|-i⟩': np.array([1, -1j], dtype=complex) / np.sqrt(2),
    }
    return states

def compute_fidelity(state1: np.ndarray, state2: np.ndarray) -> float:
    """
    Compute fidelity between two quantum states.
    Fidelity = |⟨ψ|φ⟩|²
    """
    return abs(np.dot(np.conj(state1), state2))

def compute_clifford_fidelities(theta: float, phi: float) -> Dict[str, float]:
    """
    Compute fidelities of |θ,φ⟩ with all 6 Clifford basis states.
    
    Args:
        theta: Polar angle
        phi: Azimuthal angle
    
    Returns:
        Dictionary mapping Clifford state names to fidelities
    """
    # Create the Bloch state
    bloch_state = create_bloch_state(theta, phi)
    
    # Get Clifford basis states
    clifford_states = get_clifford_basis_states()
    
    # Compute fidelities
    fidelities = {}
    for name, clifford_state in clifford_states.items():
        fid = compute_fidelity(bloch_state, clifford_state)
        fidelities[name] = fid
    
    return fidelities

def find_minimum_clifford_distance(theta: float, phi: float) -> Tuple[str, float]:
    """
    Find the Clifford state with minimum fidelity to |θ,φ⟩.
    
    Returns:
        (state_name, min_fidelity)
    """
    fidelities = compute_clifford_fidelities(theta, phi)
    print(fidelities)
    
    # Find minimum
    min_state = min(fidelities, key=fidelities.get)
    min_fidelity = fidelities[min_state]
    
    return min_state, min_fidelity

def analyze_state(theta: float, phi: float):
    """
    Complete analysis showing all fidelities and the minimum.
    """
    # Create the state
    state = create_bloch_state(theta, phi)
    
    print(f"Bloch Sphere State: |θ={theta:.4f}, φ={phi:.4f}⟩")
    print(f"State vector: [{state[0]:.6f}, {state[1]:.6f}]")
    print(f"{'='*70}")
    
    # Compute fidelities
    fidelities = compute_clifford_fidelities(theta, phi)
    
    # Find minimum
    min_state, min_fidelity = find_minimum_clifford_distance(theta, phi)
    
    print(f"Fidelities with Clifford Basis States:")
    print(f"{'-'*70}")
    
    # Sort by fidelity (highest to lowest)
    sorted_fidelities = sorted(fidelities.items(), key=lambda x: x[1], reverse=True)
    
    for name, fid in sorted_fidelities:
        marker = " ← MINIMUM" if name == min_state else ""
        print(f"{name:6s}: {fid:.10f}{marker}")
    
    print(f"{'-'*70}")
    print(f"Minimum fidelity: {min_state} with fidelity = {min_fidelity:.10f}")
    print(f"{'='*70}")
    
    return fidelities, min_state, min_fidelity

# Example usage
if __name__ == "__main__":
    # Example 1: A general state
    print("Example 1: General State")
    theta1 = np.pi / 3
    phi1 = np.pi / 4
    analyze_state(theta1, phi1)
    
    print("\n" * 2)
    
    # Example 2: The |+⟩ state (should have fidelity 1 with |+⟩)
    print("Example 2: |+⟩ State (θ=π/2, φ=0)")
    theta2 = np.pi / 2
    phi2 = 0
    analyze_state(theta2, phi2)
    
    print("\n" * 2)
    
    # Example 3: The |i⟩ state (should have fidelity 1 with |i⟩)
    print("Example 3: |i⟩ State (θ=π/2, φ=π/2)")
    theta3 = np.pi / 2
    phi3 = np.pi / 2
    analyze_state(theta3, phi3)
    
    print("\n" * 2)
    
    # Example 4: A magic state (low overlap with all Clifford states)
    print("Example 4: Magic State (θ=π/4, φ=π/8)")
    theta4 = np.pi / 4
    phi4 = np.pi / 8
    analyze_state(theta4, phi4)
    
    print("\n" * 2)
    
    # Example 5: Just get the minimum
    print("Example 5: Quick minimum fidelity check")
    theta5 = 2 * np.pi / 5
    phi5 = 3 * np.pi / 7
    min_state, min_fid = find_minimum_clifford_distance(theta5, phi5)
    print(f"For θ={theta5:.4f}, φ={phi5:.4f}:")
    print(f"Minimum Clifford fidelity: {min_state} = {min_fid:.10f}")

Example 1: General State
Bloch Sphere State: |θ=1.0472, φ=0.7854⟩
State vector: [0.866025+0.000000j, 0.353553+0.353553j]
{'|0⟩': np.float64(0.8660254037844387), '|1⟩': np.float64(0.4999999999999999), '|+⟩': np.float64(0.8978787322617109), '|-⟩': np.float64(0.44024286723591866), '|i⟩': np.float64(0.8978787322617109), '|-i⟩': np.float64(0.4402428672359187)}


TypeError: loop of ufunc does not support argument 0 of type dict which has no callable arccos method